In [8]:
#!pip install openai --upgrade
#!pip install langchain --upgrade
#!pip install llama-index --upgrade
#!pip install python-dotenv

In [6]:
import os
import openai
from dotenv import load_dotenv
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, PromptHelper, LangchainEmbedding, ServiceContext
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [7]:


# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_version = os.getenv('OPENAI_API_VERSION')
# openai.log = "debug"

In [12]:


# Initialize LLM and Embeddings model (model is the actual model name, e.g., gpt-35-turbo, engine is your custom deployment name, e.g., my-gpt-35-turbo)
llm = AzureOpenAI(engine="loveadagpt3516k_2", model="gpt-35-turbo-16k", temperature=0.0)
embeddings = LangchainEmbedding(OpenAIEmbeddings(deployment_id="loveadaembed2", chunk_size=1))

# Define prompt helper
prompt_helper = PromptHelper(context_window=3000, 
                             num_output=500, 
                             chunk_overlap_ratio=0.1, 
                             chunk_size_limit=1000)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, prompt_helper=prompt_helper)

# Load documents
documents = SimpleDirectoryReader('data/').load_data()

# Create index
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, prompt_helper=prompt_helper)
query_engine = index.as_query_engine(service_context=service_context, verbose=True)

query = "What are the errors in the data?"
answer = query_engine.query(query)

print(f"Query: {query}")
print(f"Answer: {answer}")
print(f"Sources: {answer.get_formatted_sources()}")

Query: What are the errors in the data?
Answer: There are no errors in the data.
Sources: > Source (Doc id: f697cb04-5b8d-41f8-8f5c-a40f78f8274f): might be system one
-- Library SevenZip not found in package, might be system one
-- Library File...

> Source (Doc id: 6ac902c3-808d-426e-9754-07687af93c2d): TrussTools/1.8.101@keysight/stable: WARN: Lib folder doesn't exist, can't collect libraries: C:\....
